In [1]:
# links to websites
territoryLink='https://www.citypopulation.de/en/world/bymap/landarea/'
birthRateLink='https://www.cia.gov/the-world-factbook/about/archives/2021/field/birth-rate/country-comparison'
energyLink='https://www.worldometers.info/energy/'

# scrapping into LIST of Dataframes

import pandas as pd

territoryList=pd.read_html(territoryLink,header=0,flavor='bs4')
birthRateList=pd.read_html(birthRateLink,header=0,flavor='bs4')
energyList=pd.read_html(energyLink,header=0,flavor='bs4')

In [2]:
# getting the Dataframe from list
territory=territoryList[0]
birthRate=birthRateList[0]
energy=energyList[0]

In [3]:
territory.shape,birthRate.shape,energy.shape

((255, 5), (227, 4), (209, 5))

In [4]:
territory.columns,birthRate.columns,energy.columns

(Index(['Rank (total)', 'Country', 'Total (square km)', 'per Capita (square m)',
        'Date'],
       dtype='object'),
 Index(['Rank', 'Country', 'births/1,000 population', 'Date of Information'], dtype='object'),
 Index(['#', 'Country', 'Energy  Consumption (BTU)', 'World  Share',
        'Per capita Yearly BTU'],
       dtype='object'))

In [5]:
# how many resulting rows after inner merging
territory.merge(birthRate,how='inner',left_on='Country',right_on='Country').shape

(207, 8)

In [6]:
# Apply fuzzy merging

In [7]:
# Countries in 'territory' but NOT in 'birthRate' 
onlyTerritory=set(territory.Country)-set(birthRate.Country)
onlyTerritory

{'Antarctica',
 'Ashmore and Cartier Islands',
 'Bahamas',
 'Bouvet Island',
 'British Indian Ocean Territory',
 'Cape Verde',
 'Caribbean Netherlands',
 'Clipperton Island',
 'Congo (Democratic Republic)',
 'Congo (Republic)',
 'Coral Sea Islands',
 'Czech Republic',
 "Côte d'Ivoire (Ivory Coast)",
 'East Timor',
 'Falkland Islands',
 'French Guiana',
 'French Southern and Antarctic Lands',
 'Gambia',
 'Guadeloupe',
 'Heard Island and McDonald Islands',
 'Jan Mayen',
 'Korea (North)',
 'Korea (South)',
 'Martinique',
 'Mayotte',
 'Micronesia',
 'Myanmar (Burma)',
 'Navassa Island',
 'Niue',
 'North Macedonia',
 'Palestinian Territories',
 'Paracel Islands',
 'Pitcairn',
 'Réunion',
 'Saint Helena, Ascension and Tristan da Cunha',
 'South Georgia and the South Sandwich Islands',
 'Spratly Islands',
 'Svalbard',
 'São Tomé and Príncipe',
 'Tokelau',
 'U.S. Pacific Island Wildlife Refuges',
 'United Kingdom of Great Britain and Northern Ireland',
 'United States of America',
 'Vatican Ci

In [8]:
# Countries in 'birthRate' but NOT in 'territory' 
onlyBirthRate=set(birthRate.Country)-set(territory.Country)
onlyBirthRate

{'Bahamas, The',
 'Burma',
 'Cabo Verde',
 'Congo, Democratic Republic of the',
 'Congo, Republic of the',
 "Cote d'Ivoire",
 'Czechia',
 'Falkland Islands (Islas Malvinas)',
 'Gambia, The',
 'Gaza Strip',
 'Korea, North',
 'Korea, South',
 'Micronesia, Federated States of',
 'Saint Helena, Ascension, and Tristan da Cunha',
 'Sao Tome and Principe',
 'Timor-Leste',
 'United Kingdom',
 'United States',
 'Virgin Islands',
 'West Bank'}

In [9]:
from thefuzz import process as fz
# take a country from onlyBirthRate
# look for a country in onlyTerritory and return the most similar
[(f,fz.extractOne(f, onlyTerritory)) for f in sorted(onlyBirthRate)]

[('Bahamas, The', ('Bahamas', 90)),
 ('Burma', ('Myanmar (Burma)', 90)),
 ('Cabo Verde', ('Cape Verde', 80)),
 ('Congo, Democratic Republic of the', ('Congo (Democratic Republic)', 95)),
 ('Congo, Republic of the', ('Congo (Republic)', 95)),
 ("Cote d'Ivoire", ("Côte d'Ivoire (Ivory Coast)", 86)),
 ('Czechia', ('Czech Republic', 64)),
 ('Falkland Islands (Islas Malvinas)', ('Falkland Islands', 90)),
 ('Gambia, The', ('Gambia', 90)),
 ('Gaza Strip', ('Western Sahara', 48)),
 ('Korea, North', ('Korea (North)', 100)),
 ('Korea, South', ('Korea (South)', 100)),
 ('Micronesia, Federated States of', ('Micronesia', 90)),
 ('Saint Helena, Ascension, and Tristan da Cunha',
  ('Saint Helena, Ascension and Tristan da Cunha', 99)),
 ('Sao Tome and Principe', ('São Tomé and Príncipe', 92)),
 ('Timor-Leste', ('East Timor', 82)),
 ('United Kingdom',
  ('United Kingdom of Great Britain and Northern Ireland', 90)),
 ('United States', ('United States of America', 90)),
 ('Virgin Islands', ('Virgin Islan

In [10]:
[(f,fz.extractOne(f, onlyTerritory)) for f in sorted(onlyBirthRate)
 if fz.extractOne(f, onlyTerritory)[1]>=87]

[('Bahamas, The', ('Bahamas', 90)),
 ('Burma', ('Myanmar (Burma)', 90)),
 ('Congo, Democratic Republic of the', ('Congo (Democratic Republic)', 95)),
 ('Congo, Republic of the', ('Congo (Republic)', 95)),
 ('Falkland Islands (Islas Malvinas)', ('Falkland Islands', 90)),
 ('Gambia, The', ('Gambia', 90)),
 ('Korea, North', ('Korea (North)', 100)),
 ('Korea, South', ('Korea (South)', 100)),
 ('Micronesia, Federated States of', ('Micronesia', 90)),
 ('Saint Helena, Ascension, and Tristan da Cunha',
  ('Saint Helena, Ascension and Tristan da Cunha', 99)),
 ('Sao Tome and Principe', ('São Tomé and Príncipe', 92)),
 ('United Kingdom',
  ('United Kingdom of Great Britain and Northern Ireland', 90)),
 ('United States', ('United States of America', 90)),
 ('Virgin Islands', ('Virgin Islands of the U.S.', 90))]

In [12]:
# Once you have good matches, you have to create dictionary like this:
changesBirthRate={f:fz.extractOne(f, onlyTerritory)[0] 
                 for f in sorted(onlyBirthRate)
                 if fz.extractOne(f, onlyTerritory)[1] >=87}
#dict of matches
changesBirthRate

{'Bahamas, The': 'Bahamas',
 'Burma': 'Myanmar (Burma)',
 'Congo, Democratic Republic of the': 'Congo (Democratic Republic)',
 'Congo, Republic of the': 'Congo (Republic)',
 'Falkland Islands (Islas Malvinas)': 'Falkland Islands',
 'Gambia, The': 'Gambia',
 'Korea, North': 'Korea (North)',
 'Korea, South': 'Korea (South)',
 'Micronesia, Federated States of': 'Micronesia',
 'Saint Helena, Ascension, and Tristan da Cunha': 'Saint Helena, Ascension and Tristan da Cunha',
 'Sao Tome and Principe': 'São Tomé and Príncipe',
 'United Kingdom': 'United Kingdom of Great Britain and Northern Ireland',
 'United States': 'United States of America',
 'Virgin Islands': 'Virgin Islands of the U.S.'}

In [13]:
# Use that dict for the replacements:
birthRate.Country.replace(to_replace=changesBirthRate,inplace=True)

In [14]:
# second try
onlyTerritory=set(territory.Country)-set(birthRate.Country)
onlyBirthRate=set(birthRate.Country)-set(territory.Country)
[(f,fz.extractOne(f, onlyTerritory)) for f in sorted(onlyBirthRate)]

[('Cabo Verde', ('Cape Verde', 80)),
 ("Cote d'Ivoire", ("Côte d'Ivoire (Ivory Coast)", 86)),
 ('Czechia', ('Czech Republic', 64)),
 ('Gaza Strip', ('Western Sahara', 48)),
 ('Timor-Leste', ('East Timor', 82)),
 ('West Bank', ('Western Sahara', 57))]

In [15]:
# second dict of changes
# select a different threshold
changesBirthRate2={f:fz.extractOne(f, onlyTerritory)[0] 
                 for f in sorted(onlyBirthRate)
                 if 74<=fz.extractOne(f, onlyTerritory)[1]}

#dict of matches
changesBirthRate2

{'Cabo Verde': 'Cape Verde',
 "Cote d'Ivoire": "Côte d'Ivoire (Ivory Coast)",
 'Timor-Leste': 'East Timor'}

In [16]:
# make the changes
birthRate.Country.replace(to_replace=changesBirthRate2,inplace=True)

In [17]:
# third try
onlyTerritory=set(territory.Country)-set(birthRate.Country)
onlyBirthRate=set(birthRate.Country)-set(territory.Country)
[(f,fz.extractOne(f, onlyTerritory)) for f in sorted(onlyBirthRate)]

[('Czechia', ('Czech Republic', 64)),
 ('Gaza Strip', ('Western Sahara', 48)),
 ('West Bank', ('Western Sahara', 57))]

In [18]:
# third dict of changes
# select a different threshold
changesBirthRate3={f:fz.extractOne(f, onlyTerritory)[0] 
                 for f in sorted(onlyBirthRate)
                 if 54<=fz.extractOne(f, onlyTerritory)[1]}

#dict of matches
changesBirthRate3

{'Czechia': 'Czech Republic', 'West Bank': 'Western Sahara'}

In [19]:
birthRate.Country.replace(to_replace=changesBirthRate3,inplace=True)

In [20]:
territory.merge(birthRate,how='inner',left_on='Country',right_on='Country').shape

(226, 8)

In [21]:
# First merge -> between territory and birthRate
merge1=territory.merge(birthRate,left_on='Country',right_on='Country')
merge1

,Rank (total),Country,Total (square km),per Capita (square m),Date,Rank,"births/1,000 population",Date of Information
0,1.0,Russia,16377742.0,113358,2018,193,9.71,2021 est.
1,3.0,China,9326410.0,6696,2018,169,11.30,2021 est.
2,4.0,United States of America,9147593.0,27960,2018,155,12.33,2021 est.
3,5.0,Canada,9093507.0,245380,2018,190,10.21,2021 est.
4,6.0,Brazil,8358140.0,40088,2018,136,13.44,2021 est.
...,...,...,...,...,...,...,...,...
221,240.0,Tuvalu,25.6,2433,2018,54,22.97,2021 est.
222,241.0,Saint Barthelemy,25.0,2475,2018,200,9.27,2021 est.
223,243.0,Nauru,21.2,1868,2018,66,21.49,2021 est.
224,246.0,Gibraltar,6.5,188,2018,132,13.99,2021 est.


In [22]:
merge1.merge(energy,how='inner',left_on='Country',right_on='Country').shape

(184, 12)

In [23]:
# To merge "merge1" and "energy"
# Repeat the last steps
# Countries in 'merge1' but NOT in 'energy' 
onlyMerge1=set(merge1.Country)-set(energy.Country)
onlyMerge1

{'American Samoa',
 'Andorra',
 'Anguilla',
 'Cape Verde',
 'Congo (Democratic Republic)',
 'Congo (Republic)',
 'Curacao',
 'Czech Republic',
 "Côte d'Ivoire (Ivory Coast)",
 'East Timor',
 'Faroe Islands',
 'Guernsey',
 'Isle of Man',
 'Jersey',
 'Korea (North)',
 'Korea (South)',
 'Kosovo',
 'Liechtenstein',
 'Macau',
 'Marshall Islands',
 'Micronesia',
 'Monaco',
 'Montserrat',
 'Myanmar (Burma)',
 'Nauru',
 'Northern Mariana Islands',
 'Palau',
 'Saint Barthelemy',
 'Saint Helena, Ascension and Tristan da Cunha',
 'Saint Kitts and Nevis',
 'Saint Martin',
 'Saint Pierre and Miquelon',
 'Saint Vincent and the Grenadines',
 'San Marino',
 'Sint Maarten',
 'São Tomé and Príncipe',
 'Turks and Caicos Islands',
 'Tuvalu',
 'United Kingdom of Great Britain and Northern Ireland',
 'United States of America',
 'Virgin Islands of the U.S.',
 'Wallis and Futuna'}

In [24]:
# Countries in 'energy' but NOT in 'merge1' 
onlyEnergy=set(energy.Country)-set(merge1.Country)
onlyEnergy

{'Cabo Verde',
 'Congo',
 'Czech Republic (Czechia)',
 "Côte d'Ivoire",
 'DR Congo',
 'Faeroe Islands',
 'French Guiana',
 'Guadeloupe',
 'Macao',
 'Martinique',
 'Myanmar',
 'North Korea',
 'North Macedonia',
 'Réunion',
 'Saint Kitts & Nevis',
 'Saint Pierre & Miquelon',
 'Sao Tome & Principe',
 'South Korea',
 'St. Vincent & Grenadines',
 'State of Palestine',
 'Timor-Leste',
 'Turks and Caicos',
 'U.S. Virgin Islands',
 'United Kingdom',
 'United States'}

In [25]:
[(f,fz.extractOne(f, onlyMerge1)) for f in sorted(onlyEnergy)]

[('Cabo Verde', ('Cape Verde', 80)),
 ('Congo', ('Congo (Democratic Republic)', 90)),
 ('Czech Republic (Czechia)', ('Czech Republic', 90)),
 ("Côte d'Ivoire", ("Côte d'Ivoire (Ivory Coast)", 90)),
 ('DR Congo', ('Congo (Democratic Republic)', 86)),
 ('Faeroe Islands', ('Faroe Islands', 96)),
 ('French Guiana', ('Faroe Islands', 54)),
 ('Guadeloupe', ('Saint Pierre and Miquelon', 51)),
 ('Macao', ('Macau', 80)),
 ('Martinique', ('Saint Martin', 61)),
 ('Myanmar', ('Myanmar (Burma)', 90)),
 ('North Korea', ('Korea (North)', 95)),
 ('North Macedonia', ('Monaco', 60)),
 ('Réunion', ('Saint Helena, Ascension and Tristan da Cunha', 60)),
 ('Saint Kitts & Nevis', ('Saint Kitts and Nevis', 95)),
 ('Saint Pierre & Miquelon', ('Saint Pierre and Miquelon', 95)),
 ('Sao Tome & Principe', ('São Tomé and Príncipe', 81)),
 ('South Korea', ('Korea (South)', 95)),
 ('St. Vincent & Grenadines', ('Saint Vincent and the Grenadines', 87)),
 ('State of Palestine', ('Isle of Man', 86)),
 ('Timor-Leste', ('E

In [26]:
[(f,fz.extractOne(f, onlyMerge1)) for f in sorted(onlyEnergy)
 if fz.extractOne(f, onlyMerge1)[1]>=87]

[('Congo', ('Congo (Democratic Republic)', 90)),
 ('Czech Republic (Czechia)', ('Czech Republic', 90)),
 ("Côte d'Ivoire", ("Côte d'Ivoire (Ivory Coast)", 90)),
 ('Faeroe Islands', ('Faroe Islands', 96)),
 ('Myanmar', ('Myanmar (Burma)', 90)),
 ('North Korea', ('Korea (North)', 95)),
 ('Saint Kitts & Nevis', ('Saint Kitts and Nevis', 95)),
 ('Saint Pierre & Miquelon', ('Saint Pierre and Miquelon', 95)),
 ('South Korea', ('Korea (South)', 95)),
 ('St. Vincent & Grenadines', ('Saint Vincent and the Grenadines', 87)),
 ('Turks and Caicos', ('Turks and Caicos Islands', 90)),
 ('U.S. Virgin Islands', ('Virgin Islands of the U.S.', 95)),
 ('United Kingdom',
  ('United Kingdom of Great Britain and Northern Ireland', 90)),
 ('United States', ('United States of America', 90))]

In [27]:
# Once you have good matches, you have to create dictionary like this:
changesEnergy={f:fz.extractOne(f, onlyMerge1)[0] 
                 for f in sorted(onlyEnergy)
                 if fz.extractOne(f, onlyMerge1)[1] >=87}
#dict of matches
changesEnergy

{'Congo': 'Congo (Democratic Republic)',
 'Czech Republic (Czechia)': 'Czech Republic',
 "Côte d'Ivoire": "Côte d'Ivoire (Ivory Coast)",
 'Faeroe Islands': 'Faroe Islands',
 'Myanmar': 'Myanmar (Burma)',
 'North Korea': 'Korea (North)',
 'Saint Kitts & Nevis': 'Saint Kitts and Nevis',
 'Saint Pierre & Miquelon': 'Saint Pierre and Miquelon',
 'South Korea': 'Korea (South)',
 'St. Vincent & Grenadines': 'Saint Vincent and the Grenadines',
 'Turks and Caicos': 'Turks and Caicos Islands',
 'U.S. Virgin Islands': 'Virgin Islands of the U.S.',
 'United Kingdom': 'United Kingdom of Great Britain and Northern Ireland',
 'United States': 'United States of America'}

In [28]:
energy.Country.replace(to_replace=changesEnergy,inplace=True)

In [29]:
# second try
onlyMerge1=set(merge1.Country)-set(energy.Country)
onlyEnergy=set(energy.Country)-set(merge1.Country)
[(f,fz.extractOne(f, onlyMerge1)) for f in sorted(onlyEnergy)]

[('Cabo Verde', ('Cape Verde', 80)),
 ('DR Congo', ('Congo (Republic)', 86)),
 ('French Guiana', ('Northern Mariana Islands', 49)),
 ('Guadeloupe', ('Tuvalu', 50)),
 ('Macao', ('Macau', 80)),
 ('Martinique', ('Saint Martin', 61)),
 ('North Macedonia', ('Monaco', 60)),
 ('Réunion', ('Saint Helena, Ascension and Tristan da Cunha', 60)),
 ('Sao Tome & Principe', ('São Tomé and Príncipe', 81)),
 ('State of Palestine', ('Isle of Man', 86)),
 ('Timor-Leste', ('East Timor', 82))]

In [30]:
# second dict of changes
# select a different threshold
changesEnergy2={f:fz.extractOne(f, onlyMerge1)[0] 
                 for f in sorted(onlyEnergy)
                 if 74<=fz.extractOne(f, onlyMerge1)[1]}

#dict of matches
changesEnergy2

{'Cabo Verde': 'Cape Verde',
 'DR Congo': 'Congo (Republic)',
 'Macao': 'Macau',
 'Sao Tome & Principe': 'São Tomé and Príncipe',
 'State of Palestine': 'Isle of Man',
 'Timor-Leste': 'East Timor'}

In [31]:
energy.Country.replace(to_replace=changesEnergy2,inplace=True)

In [32]:
# third try
onlyMerge1=set(merge1.Country)-set(energy.Country)
onlyEnergy=set(energy.Country)-set(merge1.Country)
[(f,fz.extractOne(f, onlyMerge1)) for f in sorted(onlyEnergy)]

[('French Guiana', ('Northern Mariana Islands', 49)),
 ('Guadeloupe', ('Tuvalu', 50)),
 ('Martinique', ('Saint Martin', 61)),
 ('North Macedonia', ('Monaco', 60)),
 ('Réunion', ('Saint Helena, Ascension and Tristan da Cunha', 60))]

In [33]:
changesBirthRate3={f:fz.extractOne(f, onlyMerge1)[0] 
                 for f in sorted(onlyEnergy)
                 if 54<=fz.extractOne(f, onlyMerge1)[1]}
changesBirthRate3

{'Martinique': 'Saint Martin',
 'North Macedonia': 'Monaco',
 'Réunion': 'Saint Helena, Ascension and Tristan da Cunha'}

In [34]:
energy.Country.replace(to_replace=changesBirthRate3,inplace=True)

In [35]:
# second merge -> between merge1 and energy
merge2=merge1.merge(energy,left_on='Country',right_on='Country')
merge2

,Rank (total),Country,Total (square km),per Capita (square m),Date,Rank,"births/1,000 population",Date of Information,#,Energy Consumption (BTU),World Share,Per capita Yearly BTU
0,1.0,Russia,16377742.0,113358,2018,193,9.71,2021 est.,3,32832743720000,5.6%,225728
1,3.0,China,9326410.0,6696,2018,169,11.30,2021 est.,1,138689472800000,23.8%,98342
2,4.0,United States of America,9147593.0,27960,2018,155,12.33,2021 est.,2,97661161460000,16.8%,296130
3,5.0,Canada,9093507.0,245380,2018,190,10.21,2021 est.,6,15062124200000,2.6%,412047
4,6.0,Brazil,8358140.0,40088,2018,136,13.44,2021 est.,9,12566399560000,2.2%,60269
...,...,...,...,...,...,...,...,...,...,...,...,...
202,234.0,Saint Martin,54.4,1532,2018,128,14.20,2021 est.,157,36175538000,0.00621%,95841
203,235.0,Bermuda,53.6,839,2018,174,11.10,2021 est.,182,11252000000,0.00193%,177827
204,239.0,Macau,28.2,43,2018,196,9.41,2021 est.,147,48682524000,0.00836%,76232
205,246.0,Gibraltar,6.5,188,2018,132,13.99,2021 est.,106,192833000000,0.0331%,5914760


In [36]:
#subir el archivo entregado por el profesor
import os
country_isos=pd.read_excel(os.path.join('country_isos.xlsx'))

In [37]:
country_isos.shape

(171, 5)

In [38]:
country_isos.columns

Index(['Country', 'Officialstatename', 'InternetccTLD', 'iso2', 'iso3'], dtype='object')

In [39]:
merge2.merge(country_isos,how='inner',left_on='Country',right_on='Country').shape

(0, 16)

In [40]:
# modify the 'Country' column of the 'merge2' dataframe so that it is in UPPER LETTERS
merge2_copy = merge2.copy()
merge2_copy['Country'] = merge2_copy['Country'].str.upper()
merge2_copy

,Rank (total),Country,Total (square km),per Capita (square m),Date,Rank,"births/1,000 population",Date of Information,#,Energy Consumption (BTU),World Share,Per capita Yearly BTU
0,1.0,RUSSIA,16377742.0,113358,2018,193,9.71,2021 est.,3,32832743720000,5.6%,225728
1,3.0,CHINA,9326410.0,6696,2018,169,11.30,2021 est.,1,138689472800000,23.8%,98342
2,4.0,UNITED STATES OF AMERICA,9147593.0,27960,2018,155,12.33,2021 est.,2,97661161460000,16.8%,296130
3,5.0,CANADA,9093507.0,245380,2018,190,10.21,2021 est.,6,15062124200000,2.6%,412047
4,6.0,BRAZIL,8358140.0,40088,2018,136,13.44,2021 est.,9,12566399560000,2.2%,60269
...,...,...,...,...,...,...,...,...,...,...,...,...
202,234.0,SAINT MARTIN,54.4,1532,2018,128,14.20,2021 est.,157,36175538000,0.00621%,95841
203,235.0,BERMUDA,53.6,839,2018,174,11.10,2021 est.,182,11252000000,0.00193%,177827
204,239.0,MACAU,28.2,43,2018,196,9.41,2021 est.,147,48682524000,0.00836%,76232
205,246.0,GIBRALTAR,6.5,188,2018,132,13.99,2021 est.,106,192833000000,0.0331%,5914760


In [41]:
merge2_copy.merge(country_isos,how='inner',left_on='Country',right_on='Country').shape

(140, 16)

In [42]:
#Fuzzy merging
onlyMerge2=set(merge2_copy.Country)-set(country_isos.Country)
onlyMerge2

{'ARUBA',
 'BAHAMAS',
 'BERMUDA',
 'BOLIVIA',
 'BRITISH VIRGIN ISLANDS',
 'BRUNEI',
 'CAPE VERDE',
 'CAYMAN ISLANDS',
 'CENTRAL AFRICAN REPUBLIC',
 'COMOROS',
 'CONGO (DEMOCRATIC REPUBLIC)',
 'CONGO (REPUBLIC)',
 'COOK ISLANDS',
 'CZECH REPUBLIC',
 "CÔTE D'IVOIRE (IVORY COAST)",
 'DOMINICA',
 'DOMINICAN REPUBLIC',
 'EAST TIMOR',
 'ERITREA',
 'FALKLAND ISLANDS',
 'FAROE ISLANDS',
 'FRENCH POLYNESIA',
 'GAMBIA',
 'GIBRALTAR',
 'GREENLAND',
 'GUAM',
 'HONG KONG',
 'IRAN',
 'ISLE OF MAN',
 'ISRAEL',
 'KIRIBATI',
 'KOREA (NORTH)',
 'KOREA (SOUTH)',
 'LAOS',
 'MACAU',
 'MOLDOVA',
 'MONACO',
 'MYANMAR (BURMA)',
 'NETHERLANDS',
 'NEW CALEDONIA',
 'NIGER',
 'PHILIPPINES',
 'PUERTO RICO',
 'RUSSIA',
 'SAINT HELENA, ASCENSION AND TRISTAN DA CUNHA',
 'SAINT KITTS AND NEVIS',
 'SAINT LUCIA',
 'SAINT MARTIN',
 'SAINT PIERRE AND MIQUELON',
 'SAINT VINCENT AND THE GRENADINES',
 'SOMALIA',
 'SOUTH AFRICA',
 'SUDAN',
 'SYRIA',
 'SÃO TOMÉ AND PRÍNCIPE',
 'TAIWAN',
 'TANZANIA',
 'TONGA',
 'TURKS AND CAICO

In [43]:
onlyCountry_isos=set(country_isos.Country)-set(merge2_copy.Country)
onlyCountry_isos

{'BAHAMAS (THE)',
 'BOLIVIA (PLURINATIONAL STATE OF)',
 'BRUNEI DARUSSALAM',
 'CABO VERDE',
 'CENTRAL AFRICAN REPUBLIC (THE)',
 'COMOROS (THE)',
 'CONGO (THE DEMOCRATIC REPUBLIC OF THE)',
 'CONGO (THE)',
 'CZECHIA',
 "C√îTE D'IVOIRE",
 'DOMINICAN REPUBLIC (THE)',
 'GAMBIA (THE)',
 'IRAN (ISLAMIC REPUBLIC OF)',
 'KOREA (THE REPUBLIC OF)',
 "LAO PEOPLE'S DEMOCRATIC REPUBLIC (THE)",
 'MICRONESIA (FEDERATED STATES OF)',
 'MOLDOVA (THE REPUBLIC OF)',
 'MYANMAR',
 'NETHERLANDS (THE)',
 'NIGER (THE)',
 'NORTH MACEDONIA',
 'PHILIPPINES (THE)',
 'RUSSIAN FEDERATION (THE)',
 'SAO TOME AND PRINCIPE',
 'SUDAN (THE)',
 'TANZANIA, THE UNITED REPUBLIC OF',
 'TIMOR-LESTE',
 'UNITED ARAB EMIRATES (THE)',
 'UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND (THE)',
 'UNITED STATES MINOR OUTLYING ISLANDS (THE)',
 'VIET NAM'}

In [44]:
[(f,fz.extractOne(f, onlyMerge2)) for f in sorted(onlyCountry_isos)]

[('BAHAMAS (THE)', ('BAHAMAS', 90)),
 ('BOLIVIA (PLURINATIONAL STATE OF)', ('BOLIVIA', 90)),
 ('BRUNEI DARUSSALAM', ('BRUNEI', 90)),
 ('CABO VERDE', ('CAPE VERDE', 80)),
 ('CENTRAL AFRICAN REPUBLIC (THE)', ('CENTRAL AFRICAN REPUBLIC', 95)),
 ('COMOROS (THE)', ('COMOROS', 90)),
 ('CONGO (THE DEMOCRATIC REPUBLIC OF THE)',
  ('CONGO (DEMOCRATIC REPUBLIC)', 95)),
 ('CONGO (THE)', ('CONGO (DEMOCRATIC REPUBLIC)', 86)),
 ('CZECHIA', ('CZECH REPUBLIC', 64)),
 ("C√îTE D'IVOIRE", ("CÔTE D'IVOIRE (IVORY COAST)", 86)),
 ('DOMINICAN REPUBLIC (THE)', ('DOMINICAN REPUBLIC', 95)),
 ('GAMBIA (THE)', ('GAMBIA', 90)),
 ('IRAN (ISLAMIC REPUBLIC OF)', ('IRAN', 90)),
 ('KOREA (THE REPUBLIC OF)', ('ISLE OF MAN', 86)),
 ("LAO PEOPLE'S DEMOCRATIC REPUBLIC (THE)", ('DOMINICAN REPUBLIC', 86)),
 ('MICRONESIA (FEDERATED STATES OF)', ('ISLE OF MAN', 86)),
 ('MOLDOVA (THE REPUBLIC OF)', ('MOLDOVA', 90)),
 ('MYANMAR', ('MYANMAR (BURMA)', 90)),
 ('NETHERLANDS (THE)', ('NETHERLANDS', 95)),
 ('NIGER (THE)', ('NIGER', 90

In [45]:
[(f,fz.extractOne(f, onlyMerge2)) for f in sorted(onlyCountry_isos)
 if fz.extractOne(f, onlyMerge2)[1]>=87]

[('BAHAMAS (THE)', ('BAHAMAS', 90)),
 ('BOLIVIA (PLURINATIONAL STATE OF)', ('BOLIVIA', 90)),
 ('BRUNEI DARUSSALAM', ('BRUNEI', 90)),
 ('CENTRAL AFRICAN REPUBLIC (THE)', ('CENTRAL AFRICAN REPUBLIC', 95)),
 ('COMOROS (THE)', ('COMOROS', 90)),
 ('CONGO (THE DEMOCRATIC REPUBLIC OF THE)',
  ('CONGO (DEMOCRATIC REPUBLIC)', 95)),
 ('DOMINICAN REPUBLIC (THE)', ('DOMINICAN REPUBLIC', 95)),
 ('GAMBIA (THE)', ('GAMBIA', 90)),
 ('IRAN (ISLAMIC REPUBLIC OF)', ('IRAN', 90)),
 ('MOLDOVA (THE REPUBLIC OF)', ('MOLDOVA', 90)),
 ('MYANMAR', ('MYANMAR (BURMA)', 90)),
 ('NETHERLANDS (THE)', ('NETHERLANDS', 95)),
 ('NIGER (THE)', ('NIGER', 90)),
 ('PHILIPPINES (THE)', ('PHILIPPINES', 95)),
 ('RUSSIAN FEDERATION (THE)', ('RUSSIA', 90)),
 ('SAO TOME AND PRINCIPE', ('SÃO TOMÉ AND PRÍNCIPE', 92)),
 ('SUDAN (THE)', ('SUDAN', 90)),
 ('TANZANIA, THE UNITED REPUBLIC OF', ('TANZANIA', 90)),
 ('UNITED ARAB EMIRATES (THE)', ('UNITED ARAB EMIRATES', 95)),
 ('UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND (THE)',


In [46]:
changesCountry={f:fz.extractOne(f, onlyMerge2)[0] 
                 for f in sorted(onlyCountry_isos)
                 if fz.extractOne(f, onlyMerge2)[1] >=87}
changesCountry

{'BAHAMAS (THE)': 'BAHAMAS',
 'BOLIVIA (PLURINATIONAL STATE OF)': 'BOLIVIA',
 'BRUNEI DARUSSALAM': 'BRUNEI',
 'CENTRAL AFRICAN REPUBLIC (THE)': 'CENTRAL AFRICAN REPUBLIC',
 'COMOROS (THE)': 'COMOROS',
 'CONGO (THE DEMOCRATIC REPUBLIC OF THE)': 'CONGO (DEMOCRATIC REPUBLIC)',
 'DOMINICAN REPUBLIC (THE)': 'DOMINICAN REPUBLIC',
 'GAMBIA (THE)': 'GAMBIA',
 'IRAN (ISLAMIC REPUBLIC OF)': 'IRAN',
 'MOLDOVA (THE REPUBLIC OF)': 'MOLDOVA',
 'MYANMAR': 'MYANMAR (BURMA)',
 'NETHERLANDS (THE)': 'NETHERLANDS',
 'NIGER (THE)': 'NIGER',
 'PHILIPPINES (THE)': 'PHILIPPINES',
 'RUSSIAN FEDERATION (THE)': 'RUSSIA',
 'SAO TOME AND PRINCIPE': 'SÃO TOMÉ AND PRÍNCIPE',
 'SUDAN (THE)': 'SUDAN',
 'TANZANIA, THE UNITED REPUBLIC OF': 'TANZANIA',
 'UNITED ARAB EMIRATES (THE)': 'UNITED ARAB EMIRATES',
 'UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND (THE)': 'UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND',
 'VIET NAM': 'VIETNAM'}

In [47]:
country_isos.Country.replace(to_replace=changesCountry,inplace=True)

In [48]:
# second try
onlyMerge2=set(merge2_copy.Country)-set(country_isos.Country)
onlyCountry_isos=set(country_isos.Country)-set(merge2_copy.Country)
[(f,fz.extractOne(f, onlyMerge2)) for f in sorted(onlyCountry_isos)]

[('CABO VERDE', ('CAPE VERDE', 80)),
 ('CONGO (THE)', ('SAINT VINCENT AND THE GRENADINES', 86)),
 ('CZECHIA', ('CZECH REPUBLIC', 64)),
 ("C√îTE D'IVOIRE", ("CÔTE D'IVOIRE (IVORY COAST)", 86)),
 ('KOREA (THE REPUBLIC OF)', ('ISLE OF MAN', 86)),
 ("LAO PEOPLE'S DEMOCRATIC REPUBLIC (THE)", ('CZECH REPUBLIC', 86)),
 ('MICRONESIA (FEDERATED STATES OF)', ('ISLE OF MAN', 86)),
 ('NORTH MACEDONIA', ('NEW CALEDONIA', 64)),
 ('TIMOR-LESTE', ('EAST TIMOR', 82)),
 ('UNITED STATES MINOR OUTLYING ISLANDS (THE)',
  ('BRITISH VIRGIN ISLANDS', 86))]

In [49]:
changesCountry2={f:fz.extractOne(f, onlyMerge2)[0] 
                 for f in sorted(onlyCountry_isos)
                 if 74<=fz.extractOne(f, onlyMerge2)[1]}
changesCountry2

{'CABO VERDE': 'CAPE VERDE',
 'CONGO (THE)': 'SAINT VINCENT AND THE GRENADINES',
 "C√îTE D'IVOIRE": "CÔTE D'IVOIRE (IVORY COAST)",
 'KOREA (THE REPUBLIC OF)': 'ISLE OF MAN',
 "LAO PEOPLE'S DEMOCRATIC REPUBLIC (THE)": 'CZECH REPUBLIC',
 'MICRONESIA (FEDERATED STATES OF)': 'ISLE OF MAN',
 'TIMOR-LESTE': 'EAST TIMOR',
 'UNITED STATES MINOR OUTLYING ISLANDS (THE)': 'BRITISH VIRGIN ISLANDS'}

In [50]:
country_isos.Country.replace(to_replace=changesCountry2,inplace=True)

In [51]:
# third try
onlyMerge2=set(merge2_copy.Country)-set(country_isos.Country)
onlyCountry_isos=set(country_isos.Country)-set(merge2_copy.Country)
[(f,fz.extractOne(f, onlyMerge2)) for f in sorted(onlyCountry_isos)]

[('CZECHIA', ('FRENCH POLYNESIA', 43)),
 ('NORTH MACEDONIA', ('NEW CALEDONIA', 64))]

In [52]:
changesCountry3={f:fz.extractOne(f, onlyMerge2)[0] 
                 for f in sorted(onlyCountry_isos)
                 if 54<=fz.extractOne(f, onlyMerge2)[1]}
changesCountry3

{'NORTH MACEDONIA': 'NEW CALEDONIA'}

In [53]:
merge2_copy.merge(country_isos,how='inner',left_on='Country',right_on='Country').shape

(169, 16)

In [54]:
# third merge -> between merge2_copy and country_isos
merge3=country_isos.merge(merge2_copy,left_on='Country',right_on='Country')
merge3

,Country,Officialstatename,InternetccTLD,iso2,iso3,Rank (total),Total (square km),per Capita (square m),Date,Rank,"births/1,000 population",Date of Information,#,Energy Consumption (BTU),World Share,Per capita Yearly BTU
0,AFGHANISTAN,The Islamic Republic of Afghanistan,.af,AF,AFG,42.0,652230.0,20657,2018,13,36.08,2021 est.,119,144694375000,0.0248%,4059
1,ALBANIA,The Republic of Albania,.al,AL,ALB,148.0,27398.0,9558,2018,141,12.86,2021 est.,128,111843460000,0.0192%,38843
2,ALGERIA,The People's Democratic Republic of Algeria,.dz,DZ,DZA,11.0,2381740.0,55938,2018,76,19.24,2021 est.,36,2398447784000,0.41184%,58305
3,ANGOLA,The Republic of Angola,.ao,AO,AGO,24.0,1246700.0,42622,2018,2,42.22,2021 est.,85,362803130000,0.0623%,12010
4,ANTIGUA AND BARBUDA,Antigua and Barbuda,.ag,AG,ATG,205.0,442.6,4649,2018,115,15.30,2021 est.,184,10605278000,0.00182%,116389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,UZBEKISTAN,The Republic of Uzbekistan,.uz,UZ,UZB,59.0,425400.0,12908,2018,109,15.85,2021 est.,42,1798254680000,0.30878%,56291
165,VIETNAM,The Socialist Republic of Viet Nam,.vn,VN,VNM,68.0,310070.0,3258,2018,106,16.04,2021 est.,32,3242513713000,0.55678%,34483
166,YEMEN,The Republic of Yemen,.ye,YE,YEM,51.0,527968.0,18417,2018,47,25.21,2021 est.,120,138496775000,0.0238%,4611
167,ZAMBIA,The Republic of Zambia,.zm,ZM,ZMB,40.0,743398.0,44020,2018,17,35.23,2021 est.,110,177977721000,0.0306%,10289


In [55]:
#cleaning data
merge3.columns=merge3.columns.str.replace(' ','_')
merge3

,Country,Officialstatename,InternetccTLD,iso2,iso3,Rank_(total),Total_(square_km),per_Capita_(square_m),Date,Rank,"births/1,000_population",Date_of_Information,#,Energy__Consumption_(BTU),World__Share,Per_capita_Yearly_BTU
0,AFGHANISTAN,The Islamic Republic of Afghanistan,.af,AF,AFG,42.0,652230.0,20657,2018,13,36.08,2021 est.,119,144694375000,0.0248%,4059
1,ALBANIA,The Republic of Albania,.al,AL,ALB,148.0,27398.0,9558,2018,141,12.86,2021 est.,128,111843460000,0.0192%,38843
2,ALGERIA,The People's Democratic Republic of Algeria,.dz,DZ,DZA,11.0,2381740.0,55938,2018,76,19.24,2021 est.,36,2398447784000,0.41184%,58305
3,ANGOLA,The Republic of Angola,.ao,AO,AGO,24.0,1246700.0,42622,2018,2,42.22,2021 est.,85,362803130000,0.0623%,12010
4,ANTIGUA AND BARBUDA,Antigua and Barbuda,.ag,AG,ATG,205.0,442.6,4649,2018,115,15.30,2021 est.,184,10605278000,0.00182%,116389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,UZBEKISTAN,The Republic of Uzbekistan,.uz,UZ,UZB,59.0,425400.0,12908,2018,109,15.85,2021 est.,42,1798254680000,0.30878%,56291
165,VIETNAM,The Socialist Republic of Viet Nam,.vn,VN,VNM,68.0,310070.0,3258,2018,106,16.04,2021 est.,32,3242513713000,0.55678%,34483
166,YEMEN,The Republic of Yemen,.ye,YE,YEM,51.0,527968.0,18417,2018,47,25.21,2021 est.,120,138496775000,0.0238%,4611
167,ZAMBIA,The Republic of Zambia,.zm,ZM,ZMB,40.0,743398.0,44020,2018,17,35.23,2021 est.,110,177977721000,0.0306%,10289


In [58]:
merge3A=merge3.drop(columns=['Rank_(total)','Rank','#','World__Share','Date'])
merge3A

,Country,Officialstatename,InternetccTLD,iso2,iso3,Total_(square_km),per_Capita_(square_m),"births/1,000_population",Date_of_Information,Energy__Consumption_(BTU),Per_capita_Yearly_BTU
0,AFGHANISTAN,The Islamic Republic of Afghanistan,.af,AF,AFG,652230.0,20657,36.08,2021 est.,144694375000,4059
1,ALBANIA,The Republic of Albania,.al,AL,ALB,27398.0,9558,12.86,2021 est.,111843460000,38843
2,ALGERIA,The People's Democratic Republic of Algeria,.dz,DZ,DZA,2381740.0,55938,19.24,2021 est.,2398447784000,58305
3,ANGOLA,The Republic of Angola,.ao,AO,AGO,1246700.0,42622,42.22,2021 est.,362803130000,12010
4,ANTIGUA AND BARBUDA,Antigua and Barbuda,.ag,AG,ATG,442.6,4649,15.30,2021 est.,10605278000,116389
...,...,...,...,...,...,...,...,...,...,...,...
164,UZBEKISTAN,The Republic of Uzbekistan,.uz,UZ,UZB,425400.0,12908,15.85,2021 est.,1798254680000,56291
165,VIETNAM,The Socialist Republic of Viet Nam,.vn,VN,VNM,310070.0,3258,16.04,2021 est.,3242513713000,34483
166,YEMEN,The Republic of Yemen,.ye,YE,YEM,527968.0,18417,25.21,2021 est.,138496775000,4611
167,ZAMBIA,The Republic of Zambia,.zm,ZM,ZMB,743398.0,44020,35.23,2021 est.,177977721000,10289


In [59]:
merge3B=merge3A.rename(columns={'Date_of_Information':'birthRate_date'})
merge3B

,Country,Officialstatename,InternetccTLD,iso2,iso3,Total_(square_km),per_Capita_(square_m),"births/1,000_population",birthRate_date,Energy__Consumption_(BTU),Per_capita_Yearly_BTU
0,AFGHANISTAN,The Islamic Republic of Afghanistan,.af,AF,AFG,652230.0,20657,36.08,2021 est.,144694375000,4059
1,ALBANIA,The Republic of Albania,.al,AL,ALB,27398.0,9558,12.86,2021 est.,111843460000,38843
2,ALGERIA,The People's Democratic Republic of Algeria,.dz,DZ,DZA,2381740.0,55938,19.24,2021 est.,2398447784000,58305
3,ANGOLA,The Republic of Angola,.ao,AO,AGO,1246700.0,42622,42.22,2021 est.,362803130000,12010
4,ANTIGUA AND BARBUDA,Antigua and Barbuda,.ag,AG,ATG,442.6,4649,15.30,2021 est.,10605278000,116389
...,...,...,...,...,...,...,...,...,...,...,...
164,UZBEKISTAN,The Republic of Uzbekistan,.uz,UZ,UZB,425400.0,12908,15.85,2021 est.,1798254680000,56291
165,VIETNAM,The Socialist Republic of Viet Nam,.vn,VN,VNM,310070.0,3258,16.04,2021 est.,3242513713000,34483
166,YEMEN,The Republic of Yemen,.ye,YE,YEM,527968.0,18417,25.21,2021 est.,138496775000,4611
167,ZAMBIA,The Republic of Zambia,.zm,ZM,ZMB,743398.0,44020,35.23,2021 est.,177977721000,10289


In [60]:
merge3B['birthRate_date']=merge3B.birthRate_date.str.extract(pat=r'(^\w+|.$)', expand=True)
merge3B

,Country,Officialstatename,InternetccTLD,iso2,iso3,Total_(square_km),per_Capita_(square_m),"births/1,000_population",birthRate_date,Energy__Consumption_(BTU),Per_capita_Yearly_BTU
0,AFGHANISTAN,The Islamic Republic of Afghanistan,.af,AF,AFG,652230.0,20657,36.08,2021,144694375000,4059
1,ALBANIA,The Republic of Albania,.al,AL,ALB,27398.0,9558,12.86,2021,111843460000,38843
2,ALGERIA,The People's Democratic Republic of Algeria,.dz,DZ,DZA,2381740.0,55938,19.24,2021,2398447784000,58305
3,ANGOLA,The Republic of Angola,.ao,AO,AGO,1246700.0,42622,42.22,2021,362803130000,12010
4,ANTIGUA AND BARBUDA,Antigua and Barbuda,.ag,AG,ATG,442.6,4649,15.30,2021,10605278000,116389
...,...,...,...,...,...,...,...,...,...,...,...
164,UZBEKISTAN,The Republic of Uzbekistan,.uz,UZ,UZB,425400.0,12908,15.85,2021,1798254680000,56291
165,VIETNAM,The Socialist Republic of Viet Nam,.vn,VN,VNM,310070.0,3258,16.04,2021,3242513713000,34483
166,YEMEN,The Republic of Yemen,.ye,YE,YEM,527968.0,18417,25.21,2021,138496775000,4611
167,ZAMBIA,The Republic of Zambia,.zm,ZM,ZMB,743398.0,44020,35.23,2021,177977721000,10289


In [62]:
merge3B.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Country                    169 non-null    object 
 1   Officialstatename          169 non-null    object 
 2   InternetccTLD              168 non-null    object 
 3   iso2                       168 non-null    object 
 4   iso3                       169 non-null    object 
 5   Total_(square_km)          169 non-null    float64
 6   per_Capita_(square_m)      169 non-null    object 
 7   births/1,000_population    169 non-null    float64
 8   birthRate_date             169 non-null    object 
 9   Energy__Consumption_(BTU)  169 non-null    int64  
 10  Per_capita_Yearly_BTU      169 non-null    int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 14.7+ KB


In [63]:
merge3B.birthRate_date.value_counts()

birthRate_date
2021    169
Name: count, dtype: int64

In [64]:
#birthRate_date is not relevant because all the data correspond to 2021
merge3C=merge3B.drop(columns=['birthRate_date'])

FinalMerge=merge3C.rename(columns={'births/1,000_population':'births/1,000_population_in_2021'})
FinalMerge

,Country,Officialstatename,InternetccTLD,iso2,iso3,Total_(square_km),per_Capita_(square_m),"births/1,000_population_in_2021",Energy__Consumption_(BTU),Per_capita_Yearly_BTU
0,AFGHANISTAN,The Islamic Republic of Afghanistan,.af,AF,AFG,652230.0,20657,36.08,144694375000,4059
1,ALBANIA,The Republic of Albania,.al,AL,ALB,27398.0,9558,12.86,111843460000,38843
2,ALGERIA,The People's Democratic Republic of Algeria,.dz,DZ,DZA,2381740.0,55938,19.24,2398447784000,58305
3,ANGOLA,The Republic of Angola,.ao,AO,AGO,1246700.0,42622,42.22,362803130000,12010
4,ANTIGUA AND BARBUDA,Antigua and Barbuda,.ag,AG,ATG,442.6,4649,15.30,10605278000,116389
...,...,...,...,...,...,...,...,...,...,...
164,UZBEKISTAN,The Republic of Uzbekistan,.uz,UZ,UZB,425400.0,12908,15.85,1798254680000,56291
165,VIETNAM,The Socialist Republic of Viet Nam,.vn,VN,VNM,310070.0,3258,16.04,3242513713000,34483
166,YEMEN,The Republic of Yemen,.ye,YE,YEM,527968.0,18417,25.21,138496775000,4611
167,ZAMBIA,The Republic of Zambia,.zm,ZM,ZMB,743398.0,44020,35.23,177977721000,10289
